# 1. PACKAGE INSTALLATION

In [ ]:
# Install required packages
!pip install transformers 
!pip install einops 
!pip install torchvision 
!pip install torch
!pip install pillow 
!pip install accelerate 
!pip install ipywidgets

# 2. IMPORTS AND CONFIGURATIONS

In [ ]:
# Import necessary libraries
import os
import json
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig
from PIL import Image
from collections import defaultdict

# Define paths and configurations
ROOT_FOLDER = 'data/bclaws/images'
OUTPUT_FILE = 'image_descriptions.json'
SUPPORTED_FORMATS = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.svg')

# Create output file if it doesn't exist
if not os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, 'w') as f:
        json.dump({}, f)
    print(f"Created empty {OUTPUT_FILE}")

# 3. MODEL INITIALIZATION

In [ ]:
# Load the processor
processor = AutoProcessor.from_pretrained(
    'allenai/Molmo-7B-D-0924',
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    'allenai/Molmo-7B-D-0924',
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

# 4. HELPER FUNCTIONS

In [ ]:
def nested_dict():
    """Create a nested defaultdict for hierarchical storage."""
    return defaultdict(nested_dict)

def convert_defaultdict_to_dict(d):
    """Convert defaultdict to regular dict for JSON serialization."""
    if isinstance(d, defaultdict):
        d = {k: convert_defaultdict_to_dict(v) for k, v in d.items()}
    return d

def process_image(image_path):
    """Process a single image and return its description."""
    prompt = """Analyze and describe the image in detail. Address the following aspects:

1. Main Subject: Identify the primary focus or theme of the image.
2. Visual Details: Describe key colors, shapes, and significant visual elements.
3. Text Content: Summarize any visible text, highlighting main topics and important points.
4. Layout and Structure: Outline how elements are arranged and any notable patterns.
5. Symbols and Legends: Explain symbols, legends, or keys present in the image.
6. Purpose and Context: Infer the likely purpose, audience, or context of the image.

Provide a comprehensive description covering all relevant details observed in the image."""

    inputs = processor.process(
        images=[Image.open(image_path)],
        text=prompt
    )
    inputs = {k: v.to(model.device).unsqueeze(0) for k, v in inputs.items()}
    
    with torch.autocast(device_type="cpu", enabled=True, dtype=torch.bfloat16):
          output = model.generate_from_batch(
              inputs,
              GenerationConfig(max_new_tokens=2000, stop_strings="<|endoftext|>"),
              tokenizer=processor.tokenizer
          )
    
    generated_tokens = output[0,inputs['input_ids'].size(1):]
    return processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)

# 5. MAIN PROCESSING LOGIC

In [ ]:
def main():
    # Initialize results dictionary
    results = nested_dict()
    
    # Load existing descriptions if any
    try:
        with open(OUTPUT_FILE, 'r') as f:
            existing_results = json.load(f)
            # Convert existing results to nested defaultdict
            for key, value in existing_results.items():
                if isinstance(value, dict):
                    results[key].update(value)
                else:
                    results[key] = value
    except json.JSONDecodeError:
        print(f"Starting with empty results as {OUTPUT_FILE} is empty or invalid")
    
    # Process directory structure
    for dirpath, dirnames, filenames in os.walk(ROOT_FOLDER):
        # Filter for image files
        image_files = [f for f in filenames if f.lower().endswith(SUPPORTED_FORMATS)]
        
        if image_files:
            # Get relative path from root folder
            rel_path = os.path.relpath(dirpath, ROOT_FOLDER)
            
            # Navigate to correct position in results dictionary
            current_dict = results
            if rel_path != '.':
                for path_part in rel_path.split(os.sep):
                    current_dict = current_dict[path_part]
            
            # Process each image
            for filename in image_files:
                image_path = os.path.join(dirpath, filename)
                if filename not in current_dict:
                    try:
                        current_dict[filename] = process_image(image_path)
                        print(f"Processed: {image_path}")
                    except Exception as e:
                        print(f"Error processing {image_path}: {str(e)}")
                else:
                    print(f"Skipped (already processed): {image_path}")
                
                # Save after each image (checkpoint)
                with open(OUTPUT_FILE, 'w') as f:
                    json.dump(convert_defaultdict_to_dict(results), f, indent=4)

    print(f"All images processed. Results saved to '{OUTPUT_FILE}'")

# 6. EXECUTION

In [ ]:
if __name__ == "__main__":
    main()